In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import datasets, transforms
import torchvision.models as models
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.utils.data as data
import torch.utils.model_zoo as model_zoo
import os


In [2]:
TRAIN_DATA_PATH = "./data/train/"
TEST_DATA_PATH =  "./data/test/"

In [3]:
# data transform, you can add different transform methods and resize image to any size
img_size = 444
train_transform = transforms.Compose([
                                    transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor()
                                    #train_transform.Translation(),
                                    #transforms.RandomHorizontalFlip()
                                    #transforms.ToPILImage()
                                    #transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                                    ])

dataset = datasets.ImageFolder(root=TRAIN_DATA_PATH,transform=train_transform)

# spilt data into train and validation and the total number of image is 4276. You can decide the number of images
# you want to use to do training and validation.
TOTAL_SIZE = len(os.listdir(TRAIN_DATA_PATH + "/NORMAL")) + len(
    os.listdir(TRAIN_DATA_PATH + "/INFECTED")
)


# spilt your data into train and val
ratio = 0.8
train_len = round(TOTAL_SIZE * ratio)
valid_len = round(TOTAL_SIZE * (1-ratio))

train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_len, valid_len])

# you can use different batch size
train_data_loader = data.DataLoader(train_dataset, batch_size=100, shuffle=True,  num_workers=4)
val_data_loader = data.DataLoader(val_dataset, batch_size=100, shuffle=True,  num_workers=4)
print(dataset)
print(dataset.class_to_idx)

Dataset ImageFolder
    Number of datapoints: 5286
    Root Location: ./data/train/
    Transforms (if any): Compose(
                             Resize(size=(444, 444), interpolation=PIL.Image.BILINEAR)
                             ToTensor()
                         )
    Target Transforms (if any): None
{'INFECTED': 0, 'NORMAL': 1}


In [4]:
# I have written the function for you this time, but it's strongly recommended that you 
# understand how to do training and validation


def train(model, data_loader, optimizer, epoch, verbose=True):
    model.train()
    loss_avg = 0.0
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        
        # loss function, you can use other loss function if you want
        loss   = F.cross_entropy(output, target)
        loss_avg = loss.item()
        
        # do back propagation
        loss.backward()
        optimizer.step()
        verbose_step = len(data_loader) // 10
        if batch_idx % verbose_step == 0 and verbose:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(data_loader.dataset),
                100. * batch_idx / len(data_loader), loss.item()))
    return loss_avg / len(data_loader)

def valid(model, data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            valid_loss += F.cross_entropy(output, target, reduction='sum').item() # sum up batch loss
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.data.view_as(pred)).cpu().sum().item() 

        valid_loss /= len(data_loader.dataset)
        print('\nValid set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            valid_loss, correct, len(data_loader.dataset),
            100. * correct / len(data_loader.dataset)))
    return float(correct) / len(data_loader.dataset)

In [5]:
############## Build the model here ##########
class ConvNet(nn.Module):
    def __init__(self,num_classes=2):
    
        super(ConvNet, self).__init__()
        
        #self.resnet_layer = nn.Sequential(*list(model.children())[:-2])
        #
        #self.transion_layer = nn.ConvTranspose2d(2048, 2048, kernel_size=14, stride=3)
        #self.pool_layer = nn.MaxPool2d(32)  
        #self.Linear_layer = nn.Linear(2048, 8)
        
        self.layer1 = nn.Sequential(
        nn.Conv2d(3, 8, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(8),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = nn.Sequential(
        nn.Conv2d(8, 16, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(16),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer3 = nn.Sequential(
        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(32),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer4 = nn.Sequential(
        nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer5 = nn.Sequential(
        nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer6 = nn.Sequential(
        nn.Conv2d(128, 256, kernel_size=5, stride=1, padding=0),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2, stride=2))
        
        
        self.fc1 = nn.Linear(2304, 1000)
        self.fc2 = nn.Linear(1000, 500)
        self.fc3 = nn.Linear(500, 100)
        self.fc4 = nn.Linear(100, num_classes)
   
       

    def forward(self, x):
        
        #out = self.resnet_layer(x)
        #out = self.transion_layer(out)
        #out = self.pool_layer(out)
        #out = out.view(out.size(0), -1) 
        #out = self.Linear_layer(out)
        
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = self.layer5(out)
        out = self.layer6(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.fc3(out)
        out = self.fc4(out)
        
        return out




In [6]:
# using gpu if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
####################  implement your optimizer ###################################
## you can use any training methods if you want (ex:lr decay, weight decay.....)
#model = models.resnet101(pretrained=True)
#for param in model.parameters():
#    param.requires_grad = False
   
#model.fc = nn.Linear(100352, 2)
#model = model.to(device)

#model = ConvNet(resnet).to(device)
model = ConvNet(num_classes=2).to(device)

lr = 0.0001
optimizer = optim.Adam(model.parameters(), lr=lr)

# start training
epochs = 3
acc = 0.0
for epoch in range(epochs):
    model.train()
    train(model, train_data_loader, optimizer, epoch)
    accuracy = valid(model, val_data_loader)
    if accuracy > acc:
        acc = accuracy
        print("-------------saving model--------------")
        # save the model
        torch.save(model, "model.pth")

In [ ]:
test_transform = transforms.Compose([transforms.Resize((img_size,img_size)),
                                    transforms.ToTensor()
                                    #transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
                                    ])

test_data = datasets.ImageFolder(root=TEST_DATA_PATH,transform=test_transform)
test_data_loader  = data.DataLoader(test_data, batch_size=64, shuffle=False, num_workers=4) 

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# load the model so that you don't need to train the model again
test_model = torch.load("model.pth").to(device)

In [ ]:
def test(model,data_loader):
    with torch.no_grad():
        model.eval()
        valid_loss = 0
        correct = 0
        bs = test_data_loader.batch_size
        result = []
        for i, (data, target) in enumerate(test_data_loader):
            data, target = data.to(device), target.to(device)
            output = model(data)
            pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
            arr = pred.data.cpu().numpy()
            for j in range(pred.size()[0]):
                file_name = test_data.samples[i*bs+j][0].split('/')[-1]
                result.append((file_name,pred[j].cpu().numpy()[0]))
    return result

In [ ]:
result = test(test_model,test_data_loader)

# Write results to csv

In [ ]:
with open ('0610832_result.csv','w') as f:
    f.write('ID,label\n')
    for data in result:
        f.write(data[0]+','+str(data[1])+'\n')